In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

In [4]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/6.5 MB ? eta -:--:--
     ---- ----------------------------------- 0.8/6.5 MB 16.8 MB/s eta 0:00:01
     ----------------- ---------------------- 2.9/6.5 MB 30.7 MB/s eta 0:00:01
     ---------------------------------------  6.5/6.5 MB 52.3 MB/s eta 0:00:01
     ---------------------------------------- 6.5/6.5 MB 34.8 MB/s eta 0:00:00
  Using cached trio-0.22.0-py3-none-any.whl (384 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached sniffio-1.3.0-py3-none-any.whl (10 kB)
     ---------------------------------------- 0.0/58.3 kB ? eta -:--:--
     ---------------------------------------- 58.3/58.3 kB 3.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

In [5]:
!apt-get update
!apt-get install chromium chromium-driver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver


In [ ]:
#Ejemplo para hacer scrapping con Selenium
init_time = time.time()

driver = web_driver()

while True:
    driver.get('https://www.goodreads.com/book/show/2767052-the-hunger-games')
    time.sleep(1)
    soup = bs(driver.page_source, 'html.parser')    #print(soup.prettify())
    if str(soup.text).find('Rate this book') > 1:
        break
    time.sleep(2)
   
driver.quit()
print(time.time()-init_time)

16.490856170654297


In [8]:
#Ejemplo para hacer scrapping con requests
init_time = time.time()
html = requests.get("https://www.goodreads.com/book/show/2767052-the-hunger-games").text

while True:
    soup = bs(html)
    #print(soup.prettify())
    if str(soup.text).find('Rate this book') > 1:
        break
    time.sleep(2)
    html = requests.get("https://www.goodreads.com/book/show/2767052-the-hunger-games").text
  
print(time.time()-init_time)

4.218833923339844


In [35]:
import requests
from bs4 import BeautifulSoup
import time

init_time = time.time()
html = requests.get("https://www.goodreads.com/book/show/2767052-the-hunger-games").text

while True:
    soup = BeautifulSoup(html, 'html.parser')
    # Buscar el elemento que contiene el género del libro
    genre_element = soup.find('span', {'class': 'greyText'}, string=' | ')
    if genre_element is not None:
        # Obtener el género del libro a partir del siguiente elemento después de " | "
        genre = genre_element.next_sibling.strip()
        break
    time.sleep(2)
    html = requests.get("https://www.goodreads.com/book/show/2767052-the-hunger-games").text
  
print('Género:', genre)
print('Tiempo total:', time.time()-init_time)


KeyboardInterrupt: 

In [10]:
import requests
from bs4 import BeautifulSoup
import time

init_time = time.time()
html = requests.get("https://www.goodreads.com/book/show/2767052-the-hunger-games").text

while True:
    soup = BeautifulSoup(html, 'html.parser')
    # Buscar los elementos que contienen la información del género del libro
    genre_elements = soup.find_all('span', {'class': 'left', 'itemprop': 'genre'})
    if genre_elements:
        # Obtener los géneros a partir del texto de los elementos
        genres = [elem.text.strip() for elem in genre_elements]
        break
    time.sleep(2)
    html = requests.get("https://www.goodreads.com/book/show/2767052-the-hunger-games").text
  
print('Géneros:', genres)
print('Tiempo total:', time.time()-init_time)


KeyboardInterrupt: 

FUNICIONA

In [40]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
import time

# Configurar el driver de Selenium
driver = webdriver.Chrome()

# Navegar a la página del libro
driver.get("https://www.goodreads.com/book/show/2767052-the-hunger-games")

# Esperar a que aparezcan los elementos de géneros
button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".BookPageMetadataSection__genres .Button__container .Button__labelItem")))
button.click()

# Extraer los géneros
genres = [genre.text for genre in genre_elements]
button = WebDriverWait(driver, 15)
# Imprimir los géneros
print("Géneros:", genres)

# Cerrar el driver de Selenium
driver.quit()






MaxRetryError: HTTPConnectionPool(host='localhost', port=62644): Max retries exceeded with url: /session/54f63f6f78236955e5316212b41af339/element/b90fb9b0-a4c6-4b44-b49c-41017b9f92ad/text (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000024C098C4DF0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [42]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Inicializar el controlador de Chrome
driver = webdriver.Chrome()

# Navegar a la página del libro
driver.get("https://www.goodreads.com/book/show/2767052-the-hunger-games")


# Esperar a que aparezcan los elementos de géneros
#genre_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".BookPageMetadataSection__genreButton:nth-child(2) .Button__labelItem")))

# Hacer clic en el botón "More Details" si existe
try:
    button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".BookPageMetadataSection__genres .Button__container .Button__labelItem")))
    button.click()
except Exception:
    pass

# Esperar a que aparezcan los elementos de géneros
genre_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".BookPageMetadataSection__genreButton .Button__labelItem")))

# Extraer los géneros
genres = [genre.text for genre in genre_elements]

# Imprimir los géneros
print("Géneros:", genres)

# Cerrar el navegador
driver.quit()


Géneros: ['Young Adult', 'Fiction', 'Dystopia', 'Fantasy', 'Science Fiction', 'Romance', 'Adventure', 'Teen', 'Post Apocalyptic', 'Audiobook']


In [43]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Inicializar el controlador de Chrome
driver = webdriver.Chrome()

# Navegar a la página del libro
driver.get("https://www.goodreads.com/book/show/2767052-the-hunger-games")


# Esperar a que aparezcan los elementos de géneros
#genre_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".BookPageMetadataSection__genreButton:nth-child(2) .Button__labelItem")))



# Esperar a que aparezcan los elementos de géneros
genre_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".BookPageMetadataSection__genreButton .Button__labelItem")))



# Extraer los géneros
genres = [genre.text for genre in genre_elements]

# Imprimir los géneros
print("Géneros:", genres)

# Cerrar el navegador
driver.quit()

Géneros: ['Young Adult', 'Fiction', 'Dystopia', 'Fantasy', 'Science Fiction', 'Romance', 'Adventure']


In [49]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Inicializar el controlador de Chrome
driver = webdriver.Chrome()

# Navegar a la página del libro
driver.get("https://www.goodreads.com/list/best_of_year/2000?id=78.Best_Books_of_2000")


button2 = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "tr:nth-child(100) .bookTitle > span")))
button2.click()

# Esperar a que aparezcan los elementos de géneros
#genre_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".BookPageMetadataSection__genreButton:nth-child(2) .Button__labelItem")))

# Hacer clic en el botón "More Details" si existe
try:
    button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".BookPageMetadataSection__genres .Button__container .Button__labelItem")))
    button.click()
except Exception:
    pass

# Esperar a que aparezcan los elementos de géneros
genre_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".BookPageMetadataSection__genreButton .Button__labelItem")))

# Extraer los géneros
genres = [genre.text for genre in genre_elements]

# Imprimir los géneros
print("Géneros:", genres)
driver.back()
# Cerrar el navegador
driver.quit()


Géneros: ['Historical Fiction', 'Fiction', 'Historical', 'British Literature', 'Literary Fiction', '18th Century', 'Prostitution']


In [48]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Inicializar el controlador de Chrome
driver = webdriver.Chrome()

# Navegar a la página del listado de libros
driver.get("https://www.goodreads.com/list/best_of_year/2000?id=78.Best_Books_of_2000")

# Buscar todos los elementos que contienen información de los libros
book_elements = driver.find_elements_by_css_selector("tr[itemtype='http://schema.org/Book']")

# Recorrer los elementos y hacer clic en cada libro
for book in book_elements:
    # Hacer clic en el título del libro
    title_element = book.find_element_by_css_selector(".bookTitle > span")
    title_element.click()

    # Esperar a que aparezcan los elementos de géneros
    genre_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".BookPageMetadataSection__genreButton .Button__labelItem")))

    # Extraer los géneros
    genres = [genre.text for genre in genre_elements]

    # Imprimir la información del libro
    print("Título:", driver.find_element_by_css_selector(".bookTitle").text)
    print("Autor:", driver.find_element_by_css_selector(".authorName > span").text)
    print("Géneros:", genres)

    # Volver a la página anterior
    driver.back()

# Cerrar el navegador
driver.quit()


AttributeError: 'WebDriver' object has no attribute 'find_elements_by_css_selector'

In [125]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.goodreads.com/list/best_of_year/2000?id=78.Best_Books_of_2000'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
books = soup.find_all('tr', {'itemtype': 'http://schema.org/Book'})

data = []

for book in books:
    title = book.find('a', {'class': 'bookTitle'}).text.strip()
    author = book.find('a', {'class': 'authorName'}).text.strip()
   
    
    data.append({'Título': title, 'Autor': author})

dflibros = pd.DataFrame(data)

print(dflibros.head())

                                              Título               Autor
0  Harry Potter and the Goblet of Fire (Harry Pot...        J.K. Rowling
1               Angels & Demons (Robert Langdon, #1)           Dan Brown
2     A Storm of Swords (A Song of Ice and Fire, #3)  George R.R. Martin
3        The Amber Spyglass (His Dark Materials, #3)      Philip Pullman
4          The Amazing Adventures of Kavalier & Clay      Michael Chabon


funcionav3

In [199]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
data = []
for w in range(1, 7):

    url = 'https://www.goodreads.com/list/best_of_year/2000?id=78.Best_Books_of_2000&page='+str(w)+''
    response = requests.get(url)

    soup = BeautifulSoup(response.text, 'html.parser')
    books = soup.find_all('tr', {'itemtype': 'http://schema.org/Book'})

    

    for book in books:
        title = book.find('a', {'class': 'bookTitle'}).text.strip()
        author = book.find('a', {'class': 'authorName'}).text.strip()
    
        
        data.append({'Título': title, 'Autor': author})

    dflibros = pd.DataFrame(data)

print(dflibros.head())

                                              Título               Autor
0  Harry Potter and the Goblet of Fire (Harry Pot...        J.K. Rowling
1               Angels & Demons (Robert Langdon, #1)           Dan Brown
2     A Storm of Swords (A Song of Ice and Fire, #3)  George R.R. Martin
3        The Amber Spyglass (His Dark Materials, #3)      Philip Pullman
4          The Amazing Adventures of Kavalier & Clay      Michael Chabon


In [202]:
dflibros['Genero'] = None


In [171]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Inicializar el controlador de Chrome
driver = webdriver.Chrome()

# Navegar a la página del libro
driver.get("https://www.goodreads.com/list/best_of_year/2000?id=78.Best_Books_of_2000")

for i in range(100):

    button2 = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "tr:nth-child("+str(i+1)+") .bookTitle > span")))
    button2.click()

    # Hacer clic en el botón "More Details" si existe
    try:
        button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".BookPageMetadataSection__genres .Button__container .Button__labelItem")))
        button.click()
    except Exception:
        pass

    # Esperar a que aparezcan los elementos de géneros
    genre_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".BookPageMetadataSection__genreButton .Button__labelItem")))

    # Extraer los géneros
    genres = [genre.text for genre in genre_elements]
    

    dflibros.at[i, 'Genero'] = genres

  
    driver.back()


# Cerrar el navegador
driver.quit()

Genero: ['Fantasy', 'Young Adult', 'Fiction', 'Magic', 'Childrens', 'Middle Grade', 'Audiobook', 'Adventure', 'Classics', 'Science Fiction Fantasy']
Genero: ['Fiction', 'Mystery', 'Thriller', 'Mystery Thriller', 'Suspense', 'Adventure', 'Crime', 'Historical Fiction', 'Novels', 'Adult']
Genero: ['Fantasy', 'Fiction', 'Epic Fantasy', 'Science Fiction Fantasy', 'High Fantasy', 'Adult', 'Audiobook', 'Adventure', 'Dragons', 'Epic']
Genero: ['Fantasy', 'Young Adult', 'Fiction', 'Childrens', 'Adventure', 'Science Fiction', 'Audiobook', 'Science Fiction Fantasy', 'Steampunk', 'Middle Grade']
Genero: ['Fiction', 'Historical Fiction', 'Novels', 'Literature', 'Historical', 'Literary Fiction', 'Contemporary', 'Jewish', 'Comics', 'New York']
Genero: ['Nonfiction', 'Humor', 'Memoir', 'Essays', 'Short Stories', 'Comedy', 'Biography', 'Audiobook', 'Biography Memoir', 'Autobiography']
Genero: ['Nonfiction', 'Writing', 'Memoir', 'Biography', 'Reference', 'Audiobook', 'Autobiography', 'Biography Memoir',

In [144]:
genres

['Fantasy',
 'Young Adult',
 'Fiction',
 'Magic',
 'Childrens',
 'Middle Grade',
 'Audiobook',
 'Adventure',
 'Classics',
 'Science Fiction Fantasy']

In [132]:
df.loc[1, 'Genero'] = ""

In [173]:
dflibros.tail(10)
    

,Título,Autor,Genero
90,"The Duke and I (Bridgertons, #1)",Julia Quinn,"[Romance, Historical Romance, Historical Ficti..."
91,Mr. Perfect,Linda Howard,"[Romance, Romantic Suspense, Suspense, Contemp..."
92,"Lord Brocktree (Redwall, #13)",Brian Jacques,"[Fantasy, Fiction, Young Adult, Childrens, Ani..."
93,"Boundary Waters (Cork O'Connor, #2)",William Kent Krueger,"[Mystery, Fiction, Thriller, Audiobook, Myster..."
94,Bee Season,Myla Goldberg,"[Fiction, Jewish, Adult Fiction, Contemporary,..."
95,"First Lady (Wynette, Texas, #4)",Susan Elizabeth Phillips,"[Romance, Contemporary Romance, Contemporary, ..."
96,His Dark Materials Omnibus,Philip Pullman,"[Fantasy, Fiction, Young Adult, Childrens, Sci..."
97,"Set in Darkness (Inspector Rebus, #11)",Ian Rankin,"[Crime, Mystery, Fiction, Scotland, Thriller, ..."
98,"P is for Peril (Kinsey Millhone, #16)",Sue Grafton,"[Mystery, Fiction, Crime, Detective, Mystery T..."
99,"Slammerkin: A Loose Dress, A Loose Woman",Emma Donoghue,"[Historical Fiction, Fiction, Historical, Brit..."


In [159]:
df.dtypes

Título     object
Autor      object
Género    float64
Genero     object
dtype: object

funciona v3

In [247]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC




for w in range(3, 7):

# Inicializar el controlador de Chrome
    driver = webdriver.Chrome()

# Navegar a la página del libro
    driver.get("https://www.goodreads.com/list/best_of_year/2000?id=78.Best_Books_of_2000&page="+str(w)+"")

    for i in range(36, 100):

        try:
            button2 = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "tr:nth-child("+str(i+1)+") .bookTitle > span")))
            button2.click()
        except Exception:
            pass
    # Hacer clic en el botón "More Details" si existe
        try:
            button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".BookPageMetadataSection__genres .Button__container .Button__labelItem")))
            button.click()
        except Exception:
            pass

    # Esperar a que aparezcan los elementos de géneros
        try:
            genre_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".BookPageMetadataSection__genreButton .Button__labelItem")))
            genres = [genre.text for genre in genre_elements]
        except Exception:
            genres = "no"
            pass
    # Extraer los géneros
        #genres = [genre.text for genre in genre_elements]
        x=i
        if(w==2):
            x=100+i
        elif (w==3):
            x=200+i
        elif (w==4):
            x=300+i
        elif (w==5):
            x=400+i
        elif (w==6):
            x=500+i                                    
        dflibros.at[x, 'Genero'] = genres
        driver.back()
    # Cerrar el navegador
    driver.quit()    
    


In [257]:
# Guardar el archivo limpio como un nuevo archivo CSV
dflibros.to_csv('../data/BX-CSV-Dump/libros.csv', sep=';', index=False)

In [259]:
dflibros.to_csv('../data/BX-CSV-Dump/libros2.csv', index=False)